In [1]:
from flask import Flask
from flask_restful import Resource, Api, request
from flask_restful import reqparse
import numpy as np
from PIL import Image
import base64
import cv2
import sys

import tensorflow as tf

from BeobWon.predictCaptcha import PredictCaptcha as BeobWonPredictCaptcha
from MinWon24.predictCaptcha import PredictCaptcha as MinWon24PredictCaptcha


app = Flask('innoCaptcha')
api = Api(app)
class CaptchaPredict(Resource) :
    
        def __init__(self, gpu_on = True) : 
            if not gpu_on :
                os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
                     
            else :
                gpu_devices = tf.config.experimental.list_physical_devices('GPU')
                for device in gpu_devices:
                    tf.config.experimental.set_memory_growth(device, True)
            
            self.r = request
            self.predictors = {}
            self.predictors['BeobWon'] = BeobWonPredictCaptcha()
            self.predictors['MinWon24'] = MinWon24PredictCaptcha()
        def post(self) :
            try :
                # convert string of image data to uint8
                img_array = np.frombuffer(base64.b64decode(self.r.data), np.uint8)     
                # decode image
                img_array = cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)
                print(img_array.shape)
                # predict
                site = self.r.args.get('site')
                print(site)
                answer = self.predictors[site].predict(img_array)      
                return {'answer' : answer}
            except Exception as e :
                print(str(e))
                return {'error' : str(e)}
            
api.add_resource(CaptchaPredict, '/predict') 


if __name__=='__main__' :
    app.run(host='0.0.0.0',debug = False, port = 5000)

 * Serving Flask app "innoCaptcha" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(40, 120, 3)
BeobWon


127.0.0.1 - - [23/Jun/2020 10:03:02] "POST /predict?site=BeobWon HTTP/1.1" 200 -


(40, 100, 3)
MinWon24


127.0.0.1 - - [23/Jun/2020 10:03:27] "POST /predict?site=MinWon24 HTTP/1.1" 200 -
